# 문제1.
## 다음 분석 흐름을 참고하여 물음에 답하시오.

### 분석 흐름

- 필요한 라이브러리 호출

- 이미지 데이터 관련 작업
    - 이미지 데이터 전처리
    - 이미지 데이터셋 불러오기
    - 훈련/검증/테스트로 분리
    - 데이터로더 정의
    
- 모델 정의

- 옵티마이저와 손실 함수 정의

- 함수 정의
    - 모델 정확도 측정 함수
    - 훈련 데이터셋을 이용할 모델 함수
    - 검증 데이터셋을 이용할 모델 성능 측정 함수
    - 모델의 학습 시간을 측정하기 위한 함수
    
- 모델 훈련

- 모델 성능 측정

#### 4가지 유형의 환경 관련 사진을 분류하는 문제입니다.
- 데이터 출처 : https://www.kaggle.com/datasets/mahmoudreda55/satellite-image-classification

#### 7번 문제 기준 가장 성능 좋게 나온 분은 소소하게나마 스벅 깊티 보내드릴게요:)
#### 직접 제가 모델을 돌리기 어려울 수 있으니 결과가 사라지지 않게 제출 부탁드려요!

In [ ]:
# 그대로 실행해주세요!
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 1. 필요한 라이브러리 호출(0.5점)

- 그대로 실행해주세요!

In [ ]:
import copy
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as Datasets

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### 2. 이미지 데이터 관련 작업

#### 1) 이미지 데이터 전처리(1점)

- transforms.Resize() 를 이용하여 훈련 데이터와 테스트 데이터 이미지 사이즈를 자유롭게 재조정하세요(ex. (256, 256))

- transforms.RandomRotation() 을 이용하여 훈련 데이터 이미지를 주어진 값 이하로 회전시키세요(ex. 5)

- transforms.RandomHorizontalFlip() 을 이용하여 훈련 데이터 이미지에 특정 확률로 무작위 수평 반전을 적용하는 변환을 생성하세요(ex. 0.5)

- 이 외에 추가로 전처리하고 싶은 방식이 있다면 자유롭게 추가해주세요.



In [ ]:
train_transforms = transforms.Compose([
                           transforms.Resize((256,256)), # 수정
                           transforms.RandomRotation(5), # 수정
                           transforms.RandomHorizontalFlip(0.5), # 수정
                           transforms.ToTensor(),
                           transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([
                           transforms.Resize((256,256)), # 수정
                           transforms.ToTensor(),
                           transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])

#### 2) 이미지 데이터셋 불러오기(0.5점)

- 여기서는 ImageFolder을 이용하겠습니다.
- 아래 path를 수정하여 실행해주세요!

In [ ]:
train_path = '/content/drive/MyDrive/data_earth/data_earth/train' # 수정
test_path = '/content/drive/MyDrive/data_earth/data_earth/test' # 수정

train_dataset = torchvision.datasets.ImageFolder(
    train_path,
    transform=train_transforms
)

test_dataset = torchvision.datasets.ImageFolder(
    test_path,
    transform=test_transforms
)

print(len(train_dataset)), print(len(test_dataset))

2381
250


(None, None)

#### 3) 훈련, 검증, 테스트로 분리(1점)

- VALID_RATIO 값을 자유롭게 수정하여 훈련과 검증 데이터를 분할해주세요.(ex. 0.8)

- 객체 복사 중 얕은 복사(shallow copy)와 깊은 복사(deep copy)의 차이에 대해 한 문장으로 설명해주세요.

In [ ]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 2381
    Root location: /content/drive/MyDrive/data_earth/data_earth/train
    StandardTransform
Transform: Compose(
               Resize(size=(256, 256), interpolation=bilinear, max_size=None, antialias=warn)
               RandomRotation(degrees=[-5.0, 5.0], interpolation=nearest, expand=False, fill=0)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [ ]:
# 훈련과 검증 데이터 분할
VALID_RATIO = 0.8 # 수정

n_train_examples = int(len(train_dataset) * VALID_RATIO)

n_valid_examples = len(train_dataset) - n_train_examples

train_data, valid_data = torch.utils.data.random_split(train_dataset,
                                           [n_train_examples, n_valid_examples])

# 검증 데이터 전처리
valid_data = copy.deepcopy(valid_data)
valid_data.dataset.transform = test_transforms

In [ ]:
train_dataset.classes

['desert', 'green_area']

- 여기에 설명해주세요 :깊은 복사는 '실제 값'을 새로운 메모리 공간에 복사하는 것을 의미하며, 얕은 복사의 경우 주소 값을 복사한다.

#### 4) 데이터로더 정의(0.5점)

- BATCH_SIZE 를 자유롭게 수정해서 분석을 진행해주세요(ex. 64)
  - 경험상 값이 너무 작으면 분석이 오래 걸리고, 값이 너무 크면 GPU 관련 에러가 발생합니다.
  - 64로 진행하고 Epoch를 5로 설정했을 때 기준 GPU 사용 시 분석 시간 약 20분 소요되는 점 참고해주세요!


In [ ]:
BATCH_SIZE = 64 # 수정
train_iterator = torch.utils.data.DataLoader(train_data,
                                 shuffle = True,
                                 batch_size = BATCH_SIZE)

valid_iterator = torch.utils.data.DataLoader(valid_data,
                                 batch_size = BATCH_SIZE)

test_iterator = torch.utils.data.DataLoader(test_dataset,
                                batch_size = BATCH_SIZE)

### 3. 모델 정의(1.5점)

- model 객체에 배치 정규화가 적용된 VGG 모델을 불러오세요.
  - VGG가 마음에 안드신다면 세션 시간에 다루었던 다른 사전학습된 모델을 가져오셔도 상관 없습니다!
- 사전 훈련된 모델을 사용합니다.
- 분류 목적을 참고하여 마지막 완전 연결층의 out_features를 변경하세요.

In [ ]:
import torchvision.models as models
model = models.vgg16(pretrained=True).to(device)  # 배치 정규화가 적용된 VGG 모델 불러오기(사전 훈련된 모델 사용)

# 새로운 out_features 값으로 변경하기 위해 완전 연결층 대체
num_classes = 4 # 분류 목적에 알맞은 클래스 수로 변경
model.classifier[6] = nn.Linear(in_features=model.classifier[6].in_features, out_features=num_classes)

model = model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 106MB/s]


### 4. 옵티마이저와 손실 함수 정의(1.5점)

- optimizer에 옵티마이저를 자유롭게 설정해주세요!(ex. optim.Adam(model.parameters(), lr = 1e-7))

- criterion에 분석 목적에 알맞은 손실 함수를 자유롭게 설정해주세요!(ex. nn.CrossEntropyLoss())

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),lr = 1e-7)# 옵티마이저
criterion = nn.CrossEntropyLoss()# 손실 함수

criterion = criterion.to(device)

### 5. 함수 정의(0.5점)

- 모두 그대로 실행해주세요!

#### 1) 모델 정확도 측정 함수

In [ ]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

#### 2) 훈련 데이터셋을 이용할 모델 함수

In [ ]:
def train(model, iterator, optimizer, criterion, device):
    epoch_loss = 0
    epoch_acc = 0

    model.train()
    for (x, y) in iterator:
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        acc = calculate_accuracy(y_pred, y)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

#### 3) 검증 데이터셋을 이용할 모델 성능 측정 함수

In [ ]:
def evaluate(model, iterator, criterion, device):
    epoch_loss = 0
    epoch_acc = 0

    model.eval()
    with torch.no_grad():
        for (x, y) in iterator:
            x = x.to(device)
            y = y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            acc = calculate_accuracy(y_pred, y)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

#### 4) 모델의 학습 시간을 측정하기 위한 함수

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

### 6. 모델 훈련(1.5점)

- EPOCHS를 자유롭게 수정해서 분석을 진행해주세요.
- 모델 저장 경로를 수정해주세요.

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
EPOCHS = 1 # 수정
best_valid_loss = float('inf')
for epoch in range(EPOCHS):
    start_time = time.monotonic()
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(),'b1' ) # 수정

    end_time = time.monotonic()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Valid. Loss: {valid_loss:.3f} |  Valid. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 9m 46s
	Train Loss: 1.350 | Train Acc: 41.08%
	 Valid. Loss: 1.374 |  Valid. Acc: 11.91%


### 7. 모델 성능 측정(1.5점)
- 모델이 저장된 경로를 불러와 test set으로 모델 성능을 측정해주세요.

In [ ]:
model.load_state_dict(torch.load('b1')) # 모델이 저장된 경로 불러오기
test_loss, test_acc = evaluate(model, test_iterator, criterion, device)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 1.356 | Test Acc: 18.99%


# 문제 2
## ResNet 구현 코드입니다.


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import pdb

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#맥 유저는 "mps:0" 사용
device

device(type='cuda', index=0)

In [ ]:
train_dataset = torchvision.datasets.MNIST('../3주차', download=True, train = True,
                                           transform = transforms.Compose([transforms.ToTensor()]))
test_dataset = torchvision.datasets.MNIST('../3주차', download=True, train = False,
                                           transform = transforms.Compose([transforms.ToTensor()]))

100%|██████████| 9912422/9912422 [00:00<00:00, 114596146.78it/s]


Extracting ../3주차/MNIST/raw/train-images-idx3-ubyte.gz to ../3주차/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 106633533.30it/s]


Extracting ../3주차/MNIST/raw/train-labels-idx1-ubyte.gz to ../3주차/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 31519252.73it/s]


Extracting ../3주차/MNIST/raw/t10k-images-idx3-ubyte.gz to ../3주차/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 3518103.19it/s]


Extracting ../3주차/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../3주차/MNIST/raw



In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100)

In [ ]:
for i, data in enumerate(train_loader):
    inputs, labels = data
print(inputs.shape)

torch.Size([100, 1, 28, 28])


##1. 아래 그림을 참고하여 빈칸을 채워주세요.

![스크린샷 2023-08-16 오후 7.24.14.png](<attachment:스크린샷 2023-08-16 오후 7.24.14.png>)

In [ ]:
class BottleNeck(nn.Module):

    def __init__(self, d_in):
        super().__init__()
        self.bn = nn.BatchNorm2d(d_in, momentum = 0.001, eps = 0.001)
        self.elu = nn.ELU()
        self.conv_1x1down = nn.Sequential(
            nn.Conv2d(in_channels=d_in, out_channels=64, kernel_size=1, padding=0),
            nn.BatchNorm2d(64, momentum = 0.001, eps = 0.001),
            nn.ELU())

        self.conv_3x3h = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64, momentum=0.001, eps=0.001),
            nn.ELU())

        self.conv_1x1up = nn.Conv2d(in_channels=64, out_channels=d_in, kernel_size=1)

    def forward(self, x):
        out = self.bn(x)
        out = self.elu(out)
        out = self.conv_1x1down(out)
        out = self.conv_3x3h(out)
        out = self.conv_1x1up(out)
        out += x

        return out

## 2. 아래 모식도를 참고하여 forward의 빈칸을 채워주세요.

![KakaoTalk_Photo_2023-08-16-21-20-14.png](attachment:KakaoTalk_Photo_2023-08-16-21-20-14.png)

In [ ]:

class ResNet(nn.Module):
    def __init__(self, d_in=1, d_out=10, d_middle=128):
        super(ResNet, self).__init__()
        self.embedding = nn.Conv2d(d_in, d_middle, 1)

        self.layer1 = BottleNeck(d_middle)
        self.layer2 = BottleNeck(d_middle)
        self.layer3 = BottleNeck(d_middle)
        self.layer4 = BottleNeck(d_middle)

        self.to_out = nn.Linear(d_middle*28*28, d_out)

    def forward(self, x):
        out = self.embedding(x)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = torch.flatten(out, start_dim=1)
        out = self.to_out(out)

        return out

In [ ]:
model = ResNet(1,10).to(device)
print(model)

ResNet(
  (embedding): Conv2d(1, 128, kernel_size=(1, 1), stride=(1, 1))
  (layer1): BottleNeck(
    (bn): BatchNorm2d(128, eps=0.001, momentum=0.001, affine=True, track_running_stats=True)
    (elu): ELU(alpha=1.0)
    (conv_1x1down): Sequential(
      (0): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(64, eps=0.001, momentum=0.001, affine=True, track_running_stats=True)
      (2): ELU(alpha=1.0)
    )
    (conv_3x3h): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=0.001, momentum=0.001, affine=True, track_running_stats=True)
      (2): ELU(alpha=1.0)
    )
    (conv_1x1up): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
  )
  (layer2): BottleNeck(
    (bn): BatchNorm2d(128, eps=0.001, momentum=0.001, affine=True, track_running_stats=True)
    (elu): ELU(alpha=1.0)
    (conv_1x1down): Sequential(
      (0): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(6

In [ ]:
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
from torch.autograd import Variable

epochs = 3
predictions_list = []
labels_list = []
num_epoch = 0
epoch_list = []
accuracy_list = []

for epoch in range(epochs):
    print("\nEpoch ", epoch)
    # train
    print("\nTrain:")
    ave_loss = 0
    cnt = 0
    for batch, data in enumerate(train_loader):

        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        train = Variable(inputs.view(100, 1, 28, 28))
        labels = Variable(labels)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        cnt = cnt + 1
        if cnt % 100 == 0:
            print('current_loss = ', loss.item())
        ave_loss += loss.item() / len(train_loader)

    num_epoch += 1

    total = 0
    correct = 0
    print("\nValidation")
    for tests, labels in test_loader:
        test, labels = tests.to(device), labels.to(device)
        labels_list.append(labels)
        outputs = model(test)
        pred = torch.argmax(outputs, 1)
        predictions_list.append(pred)
        correct += (pred == labels).sum()
        total += len(labels)

    accuracy = correct * 100 / total
    epoch_list.append(num_epoch)
    accuracy_list.append(accuracy)
    print("# epoch = {}, loss : {}, Accuracy: {}%".format(num_epoch, ave_loss, accuracy))



Epoch  0

Train:
current_loss =  5.6277852058410645
current_loss =  9.685074806213379
current_loss =  8.74429702758789
current_loss =  5.84733772277832
current_loss =  6.9178690910339355
current_loss =  9.735209465026855

Validation
# epoch = 1, loss : 15.26620030534371, Accuracy: 89.98999786376953%

Epoch  1

Train:
current_loss =  1.023603916168213
current_loss =  1.0478816032409668
current_loss =  4.315898418426514
current_loss =  12.419900894165039
current_loss =  3.8807895183563232
current_loss =  15.031715393066406

Validation
# epoch = 2, loss : 4.799834499882336, Accuracy: 89.62999725341797%

Epoch  2

Train:
current_loss =  0.23985549807548523
current_loss =  1.4077471494674683
current_loss =  5.829556941986084
current_loss =  0.333763986825943
current_loss =  5.765289783477783
current_loss =  6.612700939178467

Validation
# epoch = 3, loss : 4.092841422622717, Accuracy: 95.55999755859375%
